# Training

Experiments were conducted using:
- `mb_vae_dti/training/run.py`: command line interface for training
- and the scripts in `scripts/training/` for running the experiments

This notebook shows some plots and analysis of the unsupervised pretraining, general DTI training and benchmark fine-tuning results.

In [5]:
from resolve import *

Setting working directory to: /home/robsyc/Desktop/thesis/MB-VAE-DTI


In [14]:
# Analyzing collected results

from mb_vae_dti.validating.analysis import *

df = load_gridsearch_results("notebooks/results/baseline_kiba_cold/")

df

top_5 = df.sort_values(by="best_val_loss", ascending=True).head(5)
print(round(get_test_averages(top_5), 4))
print(f"Params: {top_5['trainable_params'].mean() / 1000000:.1f}M")

# # # get subsets with desired distinction (encoder_type and aggregator_type)
# with_concat = df[df["config.model.aggregator_type"] == "concat"]
# with_attentive = df[df["config.model.aggregator_type"] == "attentive"]

# # # get 5 best performers
# with_concat_best = with_concat.sort_values(by="best_val_loss", ascending=True).head(5)
# with_attentive_best = with_attentive.sort_values(by="best_val_loss", ascending=True).head(5)

# # # get stats
# print(f"Params: {with_concat_best['trainable_params'].mean() / 1000000:.1f}M")
# print(round(get_test_averages(with_concat_best), 4))
# print("-" * 100)
# print(f"Params: {with_attentive_best['trainable_params'].mean() / 1000000:.1f}M")
# print(round(get_test_averages(with_attentive_best), 4))

# cols_of_interest = ["experiment_name", "trainable_params", "test/real_pKd_mse", "test/real_pKi_mse", "test/real_KIBA_mse", "test/binary_accuracy", "test/binary_f1", "test/binary_auprc"]
# df_test = df[cols_of_interest].copy()
# df_test.columns = [col.replace("test/", "") if col.startswith("test/") else col for col in df_test.columns]
# print(df_test.to_string(index=False, float_format="{:8.4f}".format))

# df = load_gridsearch_results("notebooks/results/train_cold/")
# cols_of_interest = ["experiment_name", "trainable_params", "test/real_pKd_mse", "test/real_pKi_mse", "test/real_KIBA_mse", "test/binary_accuracy", "test/binary_f1", "test/binary_auprc"]
# df_test = df[cols_of_interest].copy()
# df_test.columns = [col.replace("test/", "") if col.startswith("test/") else col for col in df_test.columns]
# print(df_test.to_string(index=False, float_format="{:8.4f}".format))

2025-08-02 02:15:29,434 - mb_vae_dti.validating.analysis - INFO - Loading 64 result files from notebooks/results/baseline_kiba_cold


test/loss              0.3702
test/Y_KIBA_ci         0.7335
test/Y_KIBA_mse        0.3702
test/Y_KIBA_pearson    0.6481
test/Y_KIBA_r2         0.3899
test/Y_KIBA_rmse       0.6084
dtype: float64
Params: 15.4M


In [99]:
df.columns

Index(['experiment_name', 'best_val_loss', 'best_epoch', 'trainable_params',
       'val/loss_Y', 'val/loss_Y_pKd', 'val/loss_Y_pKi', 'val/loss_Y_KIBA',
       'val/loss', 'val/binary_accuracy', 'val/binary_auprc',
       'val/binary_auroc', 'val/binary_f1', 'val/real_pKd_ci',
       'val/real_pKd_mse', 'val/real_pKd_pearson', 'val/real_pKd_r2',
       'val/real_pKd_rmse', 'val/real_pKi_ci', 'val/real_pKi_mse',
       'val/real_pKi_pearson', 'val/real_pKi_r2', 'val/real_pKi_rmse',
       'val/real_KIBA_ci', 'val/real_KIBA_mse', 'val/real_KIBA_pearson',
       'val/real_KIBA_r2', 'val/real_KIBA_rmse', 'val/loss_drug_contrastive',
       'val/loss_target_contrastive', 'val/loss_contrastive',
       'val/loss_accuracy', 'test/loss_Y', 'test/loss_Y_pKd',
       'test/loss_Y_pKi', 'test/loss_Y_KIBA', 'test/loss',
       'test/binary_accuracy', 'test/binary_auprc', 'test/binary_auroc',
       'test/binary_f1', 'test/real_pKd_ci', 'test/real_pKd_mse',
       'test/real_pKd_pearson', 'test/rea

In [90]:
top_5

,experiment_name,best_val_loss,best_epoch,trainable_params,val/loss,val/Y_pKd_ci,val/Y_pKd_mse,val/Y_pKd_pearson,val/Y_pKd_r2,val/Y_pKd_rmse,...,config.loss.weights,config.data.batch_size,config.data.h5_path,config.data.drug_features,config.data.target_features,config.model.embedding_dim,config.model.hidden_dim,config.model.n_layers,config.model.dropout,config.model.encoder_type
1,baseline_finetune_DAVIS_cold_b00c0028,0.562607,10,1720576,0.562607,0.761968,0.562607,0.548152,0.287945,0.922116,...,1_0_0_0,16,data/input/dti.h5torch,FP-Morgan,FP-ESP,512,128,3,0.1,resnet
34,baseline_finetune_DAVIS_cold_b01c0001,0.585412,12,1456640,0.585412,0.746837,0.585412,0.516502,0.259082,0.970364,...,1_0_0_0,16,data/input/dti.h5torch,FP-Morgan,FP-ESP,512,128,2,0.1,resnet
36,baseline_finetune_DAVIS_cold_b00c0030,0.591870,12,1838272,0.591870,0.748683,0.591870,0.517225,0.250909,1.033440,...,1_0_0_0,16,data/input/dti.h5torch,FP-Morgan,FP-ESP,768,128,3,0.1,transformer
39,baseline_finetune_DAVIS_cold_b01c0006,0.592008,44,1838272,0.592008,0.742334,0.592008,0.501378,0.250734,0.916280,...,1_0_0_0,16,data/input/dti.h5torch,FP-Morgan,FP-ESP,768,128,3,0.1,transformer
62,baseline_finetune_DAVIS_cold_b01c0023,0.595989,10,1720576,0.595989,0.774765,0.595989,0.560745,0.245695,0.918591,...,1_0_0_0,16,data/input/dti.h5torch,FP-Morgan,FP-ESP,512,128,3,0.1,resnet


In [ ]:
# Generating a molecule from the limit distribution

from mb_vae_dti.training.diffusion.utils import *
import json
import torch

json_path = "data/processed/molecular_statistics.json"

stats = json.load(open(json_path))

dataset_name = "drugs_cold"

visualization_tools = MolecularVisualization(stats["general"]["atom_types"])
nodes_dist = DistributionNodes(stats["datasets"][dataset_name]["node_count_distribution"])
limit_dist = PlaceHolder(
    X=torch.tensor(stats["datasets"][dataset_name]["node_marginals"]), 
    E=torch.tensor(stats["datasets"][dataset_name]["edge_marginals"]), 
    y=torch.ones(1) / 1
)

print(f"Limit distribution\nX: {limit_dist.X.shape}\nE: {limit_dist.E.shape}")

n_nodes = nodes_dist.sample_n(1, device="cpu")
print(f"Number of nodes: {n_nodes} (sampled)")

arange = torch.arange(max(n_nodes), device="cpu").unsqueeze(0).expand(1, -1)
node_mask = arange < n_nodes.unsqueeze(1)

print(f"Node mask: {node_mask}")

z_T = sample_discrete_feature_noise(limit_dist=limit_dist, node_mask=node_mask)

print(f"Z_T.X.shape: {z_T.X.shape}")
print(f"Z_T.E.shape: {z_T.E.shape}")

edge_types = torch.argmax(z_T.E, dim=-1)
edge_types = edge_types# - 1
node_types = torch.argmax(z_T.X, dim=-1)


limit_mol = visualization_tools.mol_from_graphs(node_types[0], edge_types[0])

print(f"Successfully created molecule: {limit_mol}")
if limit_mol is not None:
    print(f"Molecule SMILES: {Chem.MolToSmiles(limit_mol)}")
else:
    print("Failed to create valid molecule") 


import rdkit.Chem as Chem
from rdkit.Chem import Draw

Draw.MolToFile(limit_mol, "limit_mol.png")

Limit distribution
X: torch.Size([8])
E: torch.Size([5])
Number of nodes: tensor([27]) (sampled)
Node mask: tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True]])
Z_T.X.shape: torch.Size([1, 27, 8])
Z_T.E.shape: torch.Size([1, 27, 27, 5])
Successfully created molecule: <rdkit.Chem.rdchem.Mol object at 0x757c5cd5a2e0>
Molecule SMILES: C.C.CC.cc1CC(C)(o(c)CC)=n234c(-n(c)n)-c(c-2O3)n-12Oc-2(c)(c)c-4
